In [1]:
import os
import numpy as np
from keras.models import Sequential # To initialise the nn as a sequence of layers
from keras.layers import Convolution2D # To make the convolution layer for 2D images
from keras.layers import MaxPooling2D # 
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import CSVLogger
from keras.optimizers import RMSprop
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

csv=CSVLogger("4_adam_custom_loss_function.log")
#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32,(2,2),input_shape = (224,224,1), activation = 'relu',strides=2,name='convo1'))
classifier.add(Convolution2D(64,(3,3), activation = 'relu',name='convo2'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Step 1 - Convolution
classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo3'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Dropout added
classifier.add(Dropout((0.3)))

# Step 3 - Flattening
classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo4'))
# Step 4 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(BatchNormalization())
classifier.add(GlobalAveragePooling2D())
classifier.add(Dropout((0.4)))
classifier.add(Dense(20, activation = 'softmax'))

Using TensorFlow backend.


### --------------------------------------------------------------------------------------------

In [10]:
weights = []
for layer in classifier.layers:
    weights.append(np.array(layer.get_weights()))

In [11]:
weights = np.array(weights)
weights.shape

(12,)

In [12]:
weights

array([ array([ array([[[[-0.08467303,  0.02599671,  0.16493297, -0.03645198,  0.09987861,
          -0.13125315,  0.05030739, -0.13011703,  0.03888449, -0.06201556,
          -0.11017871, -0.04479676,  0.09873867,  0.10402444,  0.19549456,
          -0.06720673, -0.01195286,  0.12585783, -0.09059083, -0.15319882,
           0.11054572,  0.01159756, -0.09742074, -0.05486359, -0.09438597,
           0.12147546, -0.02242406,  0.07579845, -0.13589209,  0.06401694,
          -0.16293551, -0.03629446]],

        [[ 0.11246762,  0.04780686,  0.20688549,  0.18563455, -0.01218756,
          -0.11037029,  0.14585716,  0.09088188, -0.04860049, -0.01004416,
          -0.03625934,  0.14413998, -0.14127722, -0.1563931 ,  0.09817755,
          -0.08165377,  0.17236891, -0.10760356,  0.07657373, -0.15185221,
           0.17443606, -0.19440377,  0.18894979,  0.0435968 ,  0.09620121,
           0.02654512, -0.16160312,  0.16729745, -0.14204706,  0.08630657,
           0.09056288,  0.03220297]]],


    

In [ ]:
from keras import backend as K
def custom_loss(y_true, y_pred):
    # scale predictions so that the class probas of each sample sum to 1
    y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
    # clip to prevent NaN's and Inf's
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # calc
    loss = y_true * K.log(y_pred) * weights
    loss = -K.sum(loss, -1)
    return loss


### --------------------------------------------------------------------------------------------

In [ ]:
adam = Adam(epsilon = 0.01)
classifier.compile(optimizer = adam, loss = custom_loss, metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

curr_path = os.getcwd()
basefolder = os.path.dirname(curr_path)
train_folder = os.path.join(basefolder, "Dataset\\Train")
test_folder = os.path.join(basefolder, "Dataset\\dev")

train_set = train_datagen.flow_from_directory(train_folder,target_size=(224, 224),batch_size=32,class_mode='categorical',color_mode='grayscale')

test_set = test_datagen.flow_from_directory(test_folder,target_size=(224, 224),batch_size=32,class_mode='categorical',color_mode='grayscale')

In [ ]:
history = classifier.fit_generator(train_set,steps_per_epoch=2600,epochs=5,validation_data=test_set,validation_steps=200,callbacks=[csv],verbose=2)